In [1]:
!pip install pydicom numpy scikit-image pillow scipy SimpleITK


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [5]:
"""
Medical Image Training - v8_batch_normalization (Batch Normalization)
Configuration: 64 LapSRN channels, 5 blocks | 128 DRRN channels, 25 blocks | LeakyReLU | Batch Normalization
"""

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from tqdm import tqdm
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# CONFIGURATION
# ==============================================================================

class Config:
    VERSION = 'v8_batch_normalization'
    DATA_DIR = './preprocessed_data'
    SAVE_DIR = './trained_models_v8'
    
    EPOCHS_SR = 50
    EPOCHS_CLASS = 30
    BATCH_SIZE = 16
    LEARNING_RATE = 1e-4
    
    LAPSRN_SCALE = 4
    DRRN_SCALE = 2
    TOTAL_SCALE = 8
    
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # v8_batch_normalization specific - ADDING BATCH NORMALIZATION
    LAPSRN_CHANNELS = 64
    LAPSRN_BLOCKS = 5
    DRRN_CHANNELS = 128
    DRRN_BLOCKS = 25
    KERNEL_SIZE = 3
    ACTIVATION = 'leaky'
    BACKBONE = 'resnet50'
    USE_BATCH_NORM = True  # NEW: Enable Batch Normalization


# ==============================================================================
# DATASETS
# ==============================================================================

class SuperResolutionDataset(Dataset):
    def __init__(self, preprocessed_data_dir, hr_patch_size=64, scale_factor=4):
        self.hr_patch_size = hr_patch_size
        self.lr_patch_size = hr_patch_size // scale_factor
        self.scale_factor = scale_factor
        self.image_files = []
        
        for category in ['Normal', 'Ischemia', 'Bleeding']:
            category_path = os.path.join(preprocessed_data_dir, category, '6_Final_Stripped')
            if os.path.exists(category_path):
                for filename in os.listdir(category_path):
                    if filename.endswith('.png'):
                        self.image_files.append(os.path.join(category_path, filename))
    
    def __len__(self):
        return len(self.image_files) * 4
    
    def __getitem__(self, idx):
        img_idx = idx // 4
        img_path = self.image_files[img_idx]
        img = Image.open(img_path).convert('L')
        img_array = np.array(img, dtype=np.float32) / 255.0
        
        h, w = img_array.shape
        if h < self.hr_patch_size or w < self.hr_patch_size:
            img = Image.fromarray((img_array * 255).astype(np.uint8))
            img = img.resize((self.hr_patch_size, self.hr_patch_size), Image.BICUBIC)
            img_array = np.array(img, dtype=np.float32) / 255.0
            h, w = img_array.shape
        
        top = np.random.randint(0, max(1, h - self.hr_patch_size + 1))
        left = np.random.randint(0, max(1, w - self.hr_patch_size + 1))
        hr_patch = img_array[top:top+self.hr_patch_size, left:left+self.hr_patch_size]
        
        hr_pil = Image.fromarray((hr_patch * 255).astype(np.uint8))
        lr_pil = hr_pil.resize((self.lr_patch_size, self.lr_patch_size), Image.BICUBIC)
        lr_patch = np.array(lr_pil, dtype=np.float32) / 255.0
        
        lr_tensor = torch.from_numpy(lr_patch.copy()).unsqueeze(0).float()
        hr_tensor = torch.from_numpy(hr_patch.copy()).unsqueeze(0).float()
        
        return lr_tensor, hr_tensor


class DRRNDataset(Dataset):
    def __init__(self, preprocessed_data_dir, patch_size=64, scale_factor=2):
        self.hr_patch_size = patch_size
        self.lr_patch_size = patch_size // scale_factor
        self.scale_factor = scale_factor
        self.image_files = []
        
        for category in ['Normal', 'Ischemia', 'Bleeding']:
            category_path = os.path.join(preprocessed_data_dir, category, '6_Final_Stripped')
            if os.path.exists(category_path):
                for filename in os.listdir(category_path):
                    if filename.endswith('.png'):
                        self.image_files.append(os.path.join(category_path, filename))
    
    def __len__(self):
        return len(self.image_files) * 4
    
    def __getitem__(self, idx):
        img_idx = idx // 4
        img_path = self.image_files[img_idx]
        img = Image.open(img_path).convert('L')
        img_array = np.array(img, dtype=np.float32) / 255.0
        
        h, w = img_array.shape
        if h < self.hr_patch_size or w < self.hr_patch_size:
            img = Image.fromarray((img_array * 255).astype(np.uint8))
            img = img.resize((self.hr_patch_size, self.hr_patch_size), Image.BICUBIC)
            img_array = np.array(img, dtype=np.float32) / 255.0
            h, w = img_array.shape
        
        top = np.random.randint(0, max(1, h - self.hr_patch_size + 1))
        left = np.random.randint(0, max(1, w - self.hr_patch_size + 1))
        hr_patch = img_array[top:top+self.hr_patch_size, left:left+self.hr_patch_size]
        
        hr_pil = Image.fromarray((hr_patch * 255).astype(np.uint8))
        lr_pil = hr_pil.resize((self.lr_patch_size, self.lr_patch_size), Image.BICUBIC)
        lr_patch = np.array(lr_pil, dtype=np.float32) / 255.0
        
        lr_tensor = torch.from_numpy(lr_patch.copy()).unsqueeze(0).float()
        hr_tensor = torch.from_numpy(hr_patch.copy()).unsqueeze(0).float()
        
        return lr_tensor, hr_tensor


class ClassificationDataset(Dataset):
    def __init__(self, preprocessed_data_dir, enhance_size=224):
        self.enhance_size = enhance_size
        self.data = []
        
        category_map = {'Normal': 0, 'Ischemia': 1, 'Bleeding': 2}
        urgency_map = {'Normal': 0.1, 'Ischemia': 0.7, 'Bleeding': 0.95}
        
        for category, label in category_map.items():
            category_path = os.path.join(preprocessed_data_dir, category, '6_Final_Stripped')
            if os.path.exists(category_path):
                for filename in os.listdir(category_path):
                    if filename.endswith('.png'):
                        self.data.append({
                            'path': os.path.join(category_path, filename),
                            'label': label,
                            'urgency': urgency_map[category]
                        })
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        img = Image.open(sample['path']).convert('L')
        img = img.resize((self.enhance_size, self.enhance_size), Image.BICUBIC)
        img_array = np.array(img, dtype=np.float32) / 255.0
        img_tensor = torch.from_numpy(img_array.copy()).unsqueeze(0).float()
        
        return img_tensor, sample['label'], sample['urgency']


# ==============================================================================
# BUILDING BLOCKS WITH BATCH NORMALIZATION
# ==============================================================================

class ResidualBlock(nn.Module):
    """Residual block with Batch Normalization"""
    def __init__(self, channels, kernel_size=3, use_batch_norm=True):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.activation = nn.LeakyReLU(0.2, True)
        
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(channels)
            self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        if self.use_batch_norm:
            out = self.bn1(out)
        out = self.activation(out)
        
        out = self.conv2(out)
        if self.use_batch_norm:
            out = self.bn2(out)
        
        return self.activation(out + residual)


class RecursiveBlock(nn.Module):
    """Recursive block with Batch Normalization"""
    def __init__(self, channels, kernel_size=3, use_batch_norm=True):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.activation = nn.LeakyReLU(0.2, True)
        
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(channels)
            self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        if self.use_batch_norm:
            out = self.bn1(out)
        out = self.activation(out)
        
        out = self.conv2(out)
        if self.use_batch_norm:
            out = self.bn2(out)
        out = self.activation(out)
        
        return out + residual


# ==============================================================================
# MODELS - v8_batch_normalization
# ==============================================================================

class LapSRN(nn.Module):
    """v8_batch_normalization: LapSRN with Batch Normalization"""
    def __init__(self, scale_factor=4, num_channels=1, use_batch_norm=True):
        super().__init__()
        self.scale_factor = scale_factor
        self.num_levels = 2  # 2x2 = 4x
        ch = 64
        
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(num_channels, ch, 3, padding=1),
            nn.LeakyReLU(0.2, True)
        )
        
        self.pyramid_levels = nn.ModuleList()
        self.image_reconstruction = nn.ModuleList()
        
        for _ in range(self.num_levels):
            layers = []
            for _ in range(5):
                layers.append(ResidualBlock(ch, 3, use_batch_norm))
            layers.append(nn.ConvTranspose2d(ch, ch, 4, stride=2, padding=1))
            if use_batch_norm:
                layers.append(nn.BatchNorm2d(ch))
            layers.append(nn.LeakyReLU(0.2, True))
            
            self.pyramid_levels.append(nn.Sequential(*layers))
            self.image_reconstruction.append(nn.Conv2d(ch, num_channels, 3, padding=1))
    
    def forward(self, x):
        features = self.feature_extraction(x)
        outputs = []
        current_features = features
        
        for level_idx in range(self.num_levels):
            current_features = self.pyramid_levels[level_idx](current_features)
            img_out = self.image_reconstruction[level_idx](current_features)
            
            if level_idx > 0:
                img_out = img_out + F.interpolate(outputs[-1], scale_factor=2, mode='bilinear', align_corners=False)
            else:
                img_out = img_out + F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
            
            outputs.append(img_out)
        
        return outputs[-1], outputs


class DRRN(nn.Module):
    """v8_batch_normalization: DRRN with Batch Normalization"""
    def __init__(self, num_channels=1, scale_factor=2, use_batch_norm=True):
        super().__init__()
        self.scale_factor = scale_factor
        ch = 128
        
        self.input_conv = nn.Conv2d(num_channels, ch, 3, padding=1)
        
        self.recursive_blocks = nn.ModuleList()
        for _ in range(25):
            self.recursive_blocks.append(RecursiveBlock(ch, 3, use_batch_norm))
        
        self.fusion = nn.Sequential(
            nn.Conv2d(ch * 3, ch, 1),
            nn.LeakyReLU(0.2, True)
        )
        
        self.upsample = nn.Sequential(
            nn.Conv2d(ch, ch * 4, 3, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, True)
        )
        
        self.output_conv = nn.Sequential(
            nn.Conv2d(ch, 64, 3, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, num_channels, 3, padding=1)
        )
    
    def forward(self, x):
        input_upsampled = F.interpolate(x, scale_factor=self.scale_factor, mode='bicubic', align_corners=False)
        
        features = self.input_conv(x)
        multi_scale_features = []
        current = features
        
        collect_indices = [8, 16, 24]
        
        for idx, block in enumerate(self.recursive_blocks):
            current = block(current)
            if idx in collect_indices:
                multi_scale_features.append(current)
        
        fused = torch.cat(multi_scale_features, dim=1)
        fused = self.fusion(fused)
        upsampled = self.upsample(fused)
        output = self.output_conv(upsampled)
        
        return output + input_upsampled


class MedicalImageClassifier(nn.Module):
    """v8_batch_normalization: Standard ResNet50 classifier (same as baseline)"""
    def __init__(self, num_classes=3):
        super().__init__()
        
        from torchvision import models
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        self.classification_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        self.urgency_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
        self.feature_head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classification_head(features), self.urgency_head(features), self.feature_head(features)


# ==============================================================================
# TRAINING
# ==============================================================================

def train_model():
    config = Config()
    
    print(f"\n{'='*80}")
    print(f"TRAINING {config.VERSION.upper()}")
    print(f"{'='*80}")
    print(f"Configuration:")
    print(f"  - LapSRN: {config.LAPSRN_CHANNELS} channels, {config.LAPSRN_BLOCKS} blocks")
    print(f"  - DRRN: {config.DRRN_CHANNELS} channels, {config.DRRN_BLOCKS} blocks")
    print(f"  - Kernel: {config.KERNEL_SIZE}x{config.KERNEL_SIZE}")
    print(f"  - Activation: {config.ACTIVATION}")
    print(f"  - Backbone: {config.BACKBONE.upper()}")
    print(f"  - Batch Normalization: Enabled")
    print(f"  - Device: {config.DEVICE}")
    print(f"{'='*80}\n")
    
    version_save_dir = os.path.join(config.SAVE_DIR, config.VERSION)
    os.makedirs(version_save_dir, exist_ok=True)
    
    # Initialize models
    lapsrn = LapSRN(use_batch_norm=config.USE_BATCH_NORM).to(config.DEVICE)
    drrn = DRRN(use_batch_norm=config.USE_BATCH_NORM).to(config.DEVICE)
    classifier = MedicalImageClassifier().to(config.DEVICE)
    
    print(f"\nBatch Normalization info:")
    print(f"  - Applied after: Every convolution in residual/recursive blocks")
    print(f"  - Benefits: Better gradient flow, faster convergence, regularization")
    print(f"  - Expected: More stable training, potentially higher accuracy")
    
    # Create datasets
    sr_dataset = SuperResolutionDataset(config.DATA_DIR, hr_patch_size=64, scale_factor=4)
    drrn_dataset = DRRNDataset(config.DATA_DIR, patch_size=64, scale_factor=2)
    class_dataset = ClassificationDataset(config.DATA_DIR, enhance_size=224)
    
    # Split datasets (80/20)
    train_sr, val_sr = torch.utils.data.random_split(sr_dataset, 
        [int(0.8*len(sr_dataset)), len(sr_dataset)-int(0.8*len(sr_dataset))])
    train_drrn, val_drrn = torch.utils.data.random_split(drrn_dataset,
        [int(0.8*len(drrn_dataset)), len(drrn_dataset)-int(0.8*len(drrn_dataset))])
    train_class, val_class = torch.utils.data.random_split(class_dataset,
        [int(0.8*len(class_dataset)), len(class_dataset)-int(0.8*len(class_dataset))])
    
    # DataLoaders
    train_sr_loader = DataLoader(train_sr, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
    train_drrn_loader = DataLoader(train_drrn, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
    train_class_loader = DataLoader(train_class, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
    
    # Train LapSRN
    print("\n" + "="*80)
    print("[1/3] Training LapSRN (16x16 → 64x64, 4x upsampling) + BatchNorm")
    print("="*80)
    
    optimizer = optim.Adam(lapsrn.parameters(), lr=config.LEARNING_RATE)
    criterion = nn.L1Loss()
    best_loss = float('inf')
    
    for epoch in range(config.EPOCHS_SR):
        lapsrn.train()
        train_loss = 0
        pbar = tqdm(train_sr_loader, desc=f'Epoch {epoch+1}/{config.EPOCHS_SR}')
        
        for lr_imgs, hr_imgs in pbar:
            lr_imgs, hr_imgs = lr_imgs.to(config.DEVICE), hr_imgs.to(config.DEVICE)
            optimizer.zero_grad()
            sr_output, _ = lapsrn(lr_imgs)
            loss = criterion(sr_output, hr_imgs)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.6f}'})
        
        avg_loss = train_loss / len(train_sr_loader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(lapsrn.state_dict(), os.path.join(version_save_dir, 'lapsrn_best.pth'))
    
    print(f"✓ LapSRN training complete (best loss: {best_loss:.6f})")
    
    # Train DRRN
    print("\n" + "="*80)
    print("[2/3] Training DRRN (64x64 → 128x128, 2x upsampling) + BatchNorm")
    print("="*80)
    
    optimizer = optim.Adam(drrn.parameters(), lr=config.LEARNING_RATE)
    best_loss = float('inf')
    
    for epoch in range(config.EPOCHS_SR):
        drrn.train()
        train_loss = 0
        pbar = tqdm(train_drrn_loader, desc=f'Epoch {epoch+1}/{config.EPOCHS_SR}')
        
        for lr_imgs, hr_imgs in pbar:
            lr_imgs, hr_imgs = lr_imgs.to(config.DEVICE), hr_imgs.to(config.DEVICE)
            optimizer.zero_grad()
            sr_output = drrn(lr_imgs)
            loss = criterion(sr_output, hr_imgs)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.6f}'})
        
        avg_loss = train_loss / len(train_drrn_loader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(drrn.state_dict(), os.path.join(version_save_dir, 'drrn_best.pth'))
    
    print(f"✓ DRRN training complete (best loss: {best_loss:.6f})")
    
    # Train Classifier
    print("\n" + "="*80)
    print("[3/3] Training Classifier (128x128 → 224x224 → Classification)")
    print("="*80)
    
    optimizer = optim.Adam(classifier.parameters(), lr=config.LEARNING_RATE)
    class_criterion = nn.CrossEntropyLoss()
    urgency_criterion = nn.BCELoss()
    best_acc = 0.0
    
    for epoch in range(config.EPOCHS_CLASS):
        classifier.train()
        correct, total = 0, 0
        pbar = tqdm(train_class_loader, desc=f'Epoch {epoch+1}/{config.EPOCHS_CLASS}')
        
        for images, labels, urgency in pbar:
            images = images.to(config.DEVICE)
            labels = labels.to(config.DEVICE)
            urgency = urgency.to(config.DEVICE).unsqueeze(1).float()
            
            optimizer.zero_grad()
            class_out, urgency_out, _ = classifier(images)
            loss = class_criterion(class_out, labels) + 0.5 * urgency_criterion(urgency_out, urgency)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(class_out, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.set_postfix({'acc': f'{100*correct/total:.2f}%'})
        
        acc = 100 * correct / total
        if acc > best_acc:
            best_acc = acc
            torch.save(classifier.state_dict(), os.path.join(version_save_dir, 'classifier_best.pth'))
    
    print(f"✓ Classifier training complete (best accuracy: {best_acc:.2f}%)")
    
    # Save configuration
    config_dict = {
        'version': config.VERSION,
        'lapsrn_channels': config.LAPSRN_CHANNELS,
        'lapsrn_blocks': config.LAPSRN_BLOCKS,
        'drrn_channels': config.DRRN_CHANNELS,
        'drrn_blocks': config.DRRN_BLOCKS,
        'kernel_size': config.KERNEL_SIZE,
        'activation': config.ACTIVATION,
        'backbone': config.BACKBONE,
        'use_batch_norm': config.USE_BATCH_NORM,
        'epochs_sr': config.EPOCHS_SR,
        'epochs_class': config.EPOCHS_CLASS,
        'timestamp': datetime.now().isoformat(),
        'notes': 'Added Batch Normalization after every convolution for better training stability'
    }
    
    with open(os.path.join(version_save_dir, 'config.json'), 'w') as f:
        json.dump(config_dict, f, indent=2)
    
    print(f"\n{'='*80}")
    print("✓ ALL TRAINING COMPLETE!")
    print(f"{'='*80}")
    print(f"Models saved to: {version_save_dir}")
    print("\nPipeline: 16x16 → LapSRN(4x) → 64x64 → DRRN(2x) → 128x128 → Classifier(224x224)")
    print("\nKey difference from v1_baseline:")
    print("  - Added Batch Normalization after every convolution")
    print("  - Benefits: Better gradient flow, faster convergence")
    print("  - Expected: More stable training, potentially higher accuracy")


if __name__ == "__main__":
    train_model()


TRAINING V8_BATCH_NORMALIZATION
Configuration:
  - LapSRN: 64 channels, 5 blocks
  - DRRN: 128 channels, 25 blocks
  - Kernel: 3x3
  - Activation: leaky
  - Backbone: RESNET50
  - Batch Normalization: Enabled
  - Device: cuda


Batch Normalization info:
  - Applied after: Every convolution in residual/recursive blocks
  - Benefits: Better gradient flow, faster convergence, regularization
  - Expected: More stable training, potentially higher accuracy

[1/3] Training LapSRN (16x16 → 64x64, 4x upsampling) + BatchNorm


Epoch 1/50: 100% 1328/1328 [00:39<00:00, 33.36it/s, loss=0.017232]
Epoch 2/50: 100% 1328/1328 [00:38<00:00, 34.86it/s, loss=0.015533]
Epoch 3/50: 100% 1328/1328 [00:38<00:00, 34.84it/s, loss=0.031663]
Epoch 4/50: 100% 1328/1328 [00:38<00:00, 34.78it/s, loss=0.008327]
Epoch 5/50: 100% 1328/1328 [00:38<00:00, 34.73it/s, loss=0.023635]
Epoch 6/50: 100% 1328/1328 [00:37<00:00, 35.21it/s, loss=0.017653]
Epoch 7/50: 100% 1328/1328 [00:38<00:00, 34.71it/s, loss=0.008038]
Epoch 8/50: 100% 1328/1328 [00:38<00:00, 34.79it/s, loss=0.027934]
Epoch 9/50: 100% 1328/1328 [00:38<00:00, 34.74it/s, loss=0.011450]
Epoch 10/50: 100% 1328/1328 [00:37<00:00, 35.06it/s, loss=0.015120]
Epoch 11/50: 100% 1328/1328 [00:37<00:00, 35.69it/s, loss=0.014431]
Epoch 12/50: 100% 1328/1328 [00:37<00:00, 35.69it/s, loss=0.015358]
Epoch 13/50: 100% 1328/1328 [00:37<00:00, 34.95it/s, loss=0.010502]
Epoch 14/50: 100% 1328/1328 [00:37<00:00, 35.15it/s, loss=0.011167]
Epoch 15/50: 100% 1328/1328 [00:38<00:00, 34.90it/s, loss

✓ LapSRN training complete (best loss: 0.009940)

[2/3] Training DRRN (64x64 → 128x128, 2x upsampling) + BatchNorm


Epoch 1/50: 100% 1328/1328 [01:32<00:00, 14.39it/s, loss=0.003764]
Epoch 2/50: 100% 1328/1328 [01:32<00:00, 14.43it/s, loss=0.008238]
Epoch 3/50: 100% 1328/1328 [01:32<00:00, 14.43it/s, loss=0.009221]
Epoch 4/50: 100% 1328/1328 [01:31<00:00, 14.48it/s, loss=0.003592]
Epoch 5/50: 100% 1328/1328 [01:31<00:00, 14.59it/s, loss=0.004094]
Epoch 6/50: 100% 1328/1328 [01:31<00:00, 14.48it/s, loss=0.002036]
Epoch 7/50: 100% 1328/1328 [01:31<00:00, 14.47it/s, loss=0.003502]
Epoch 8/50: 100% 1328/1328 [01:31<00:00, 14.49it/s, loss=0.003578]
Epoch 9/50: 100% 1328/1328 [01:31<00:00, 14.45it/s, loss=0.002700]
Epoch 10/50: 100% 1328/1328 [01:31<00:00, 14.48it/s, loss=0.001680]
Epoch 11/50: 100% 1328/1328 [01:31<00:00, 14.45it/s, loss=0.001954]
Epoch 12/50: 100% 1328/1328 [01:31<00:00, 14.53it/s, loss=0.005767]
Epoch 13/50: 100% 1328/1328 [01:31<00:00, 14.46it/s, loss=0.003244]
Epoch 14/50: 100% 1328/1328 [01:31<00:00, 14.46it/s, loss=0.005096]
Epoch 15/50: 100% 1328/1328 [01:32<00:00, 14.43it/s, loss

✓ DRRN training complete (best loss: 0.002472)

[3/3] Training Classifier (128x128 → 224x224 → Classification)


Epoch 1/30: 100% 332/332 [00:22<00:00, 14.78it/s, acc=78.09%]
Epoch 2/30: 100% 332/332 [00:11<00:00, 28.38it/s, acc=86.10%]
Epoch 3/30: 100% 332/332 [00:11<00:00, 28.40it/s, acc=90.47%]
Epoch 4/30: 100% 332/332 [00:11<00:00, 28.75it/s, acc=92.56%]
Epoch 5/30: 100% 332/332 [00:11<00:00, 28.42it/s, acc=93.99%]
Epoch 6/30: 100% 332/332 [00:11<00:00, 28.42it/s, acc=95.82%]
Epoch 7/30: 100% 332/332 [00:11<00:00, 28.30it/s, acc=95.16%]
Epoch 8/30: 100% 332/332 [00:11<00:00, 28.23it/s, acc=96.02%]
Epoch 9/30: 100% 332/332 [00:11<00:00, 28.48it/s, acc=96.67%]
Epoch 10/30: 100% 332/332 [00:11<00:00, 28.54it/s, acc=95.80%]
Epoch 11/30: 100% 332/332 [00:11<00:00, 28.67it/s, acc=96.65%]
Epoch 12/30: 100% 332/332 [00:11<00:00, 27.91it/s, acc=96.55%]
Epoch 13/30: 100% 332/332 [00:11<00:00, 27.91it/s, acc=96.87%]
Epoch 14/30: 100% 332/332 [00:11<00:00, 28.59it/s, acc=97.32%]
Epoch 15/30: 100% 332/332 [00:11<00:00, 27.77it/s, acc=97.06%]
Epoch 16/30: 100% 332/332 [00:11<00:00, 28.20it/s, acc=97.04%]
E

✓ Classifier training complete (best accuracy: 98.04%)

✓ ALL TRAINING COMPLETE!
Models saved to: ./trained_models_v8/v8_batch_normalization

Pipeline: 16x16 → LapSRN(4x) → 64x64 → DRRN(2x) → 128x128 → Classifier(224x224)

Key difference from v1_baseline:
  - Added Batch Normalization after every convolution
  - Benefits: Better gradient flow, faster convergence
  - Expected: More stable training, potentially higher accuracy


In [8]:
"""
================================================================================
EVALUATION SCRIPT - v8_batch_normalization
================================================================================
Evaluates v8_batch_normalization using the SAME dataset structure as training
"""

import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# MODEL DEFINITIONS FOR V8_BATCH_NORMALIZATION
# ============================================================================

class ResidualBlock(nn.Module):
    """Residual block with Batch Normalization"""
    def __init__(self, channels, kernel_size=3, use_batch_norm=True):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.activation = nn.LeakyReLU(0.2, True)
        
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(channels)
            self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        if self.use_batch_norm:
            out = self.bn1(out)
        out = self.activation(out)
        
        out = self.conv2(out)
        if self.use_batch_norm:
            out = self.bn2(out)
        
        return self.activation(out + residual)


class RecursiveBlock(nn.Module):
    """Recursive block with Batch Normalization"""
    def __init__(self, channels, kernel_size=3, use_batch_norm=True):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.activation = nn.LeakyReLU(0.2, True)
        
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            self.bn1 = nn.BatchNorm2d(channels)
            self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        if self.use_batch_norm:
            out = self.bn1(out)
        out = self.activation(out)
        
        out = self.conv2(out)
        if self.use_batch_norm:
            out = self.bn2(out)
        out = self.activation(out)
        
        return out + residual


class LapSRN(nn.Module):
    """v8_batch_normalization: LapSRN with Batch Normalization"""
    def __init__(self, scale_factor=4, num_channels=1, use_batch_norm=True):
        super().__init__()
        self.scale_factor = scale_factor
        self.num_levels = 2  # 2x2 = 4x
        ch = 64
        
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(num_channels, ch, 3, padding=1),
            nn.LeakyReLU(0.2, True)
        )
        
        self.pyramid_levels = nn.ModuleList()
        self.image_reconstruction = nn.ModuleList()
        
        for _ in range(self.num_levels):
            layers = []
            for _ in range(5):
                layers.append(ResidualBlock(ch, 3, use_batch_norm))
            layers.append(nn.ConvTranspose2d(ch, ch, 4, stride=2, padding=1))
            if use_batch_norm:
                layers.append(nn.BatchNorm2d(ch))
            layers.append(nn.LeakyReLU(0.2, True))
            
            self.pyramid_levels.append(nn.Sequential(*layers))
            self.image_reconstruction.append(nn.Conv2d(ch, num_channels, 3, padding=1))
    
    def forward(self, x):
        features = self.feature_extraction(x)
        outputs = []
        current_features = features
        
        for level_idx in range(self.num_levels):
            current_features = self.pyramid_levels[level_idx](current_features)
            img_out = self.image_reconstruction[level_idx](current_features)
            
            if level_idx > 0:
                img_out = img_out + F.interpolate(outputs[-1], scale_factor=2, mode='bilinear', align_corners=False)
            else:
                img_out = img_out + F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
            
            outputs.append(img_out)
        
        return outputs[-1], outputs


class DRRN(nn.Module):
    """v8_batch_normalization: DRRN with Batch Normalization"""
    def __init__(self, num_channels=1, scale_factor=2, use_batch_norm=True):
        super().__init__()
        self.scale_factor = scale_factor
        ch = 128
        
        self.input_conv = nn.Conv2d(num_channels, ch, 3, padding=1)
        
        self.recursive_blocks = nn.ModuleList()
        for _ in range(25):
            self.recursive_blocks.append(RecursiveBlock(ch, 3, use_batch_norm))
        
        self.fusion = nn.Sequential(
            nn.Conv2d(ch * 3, ch, 1),
            nn.LeakyReLU(0.2, True)
        )
        
        self.upsample = nn.Sequential(
            nn.Conv2d(ch, ch * 4, 3, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, True)
        )
        
        self.output_conv = nn.Sequential(
            nn.Conv2d(ch, 64, 3, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, num_channels, 3, padding=1)
        )
    
    def forward(self, x):
        input_upsampled = F.interpolate(x, scale_factor=self.scale_factor, mode='bicubic', align_corners=False)
        
        features = self.input_conv(x)
        multi_scale_features = []
        current = features
        
        collect_indices = [8, 16, 24]
        
        for idx, block in enumerate(self.recursive_blocks):
            current = block(current)
            if idx in collect_indices:
                multi_scale_features.append(current)
        
        fused = torch.cat(multi_scale_features, dim=1)
        fused = self.fusion(fused)
        upsampled = self.upsample(fused)
        output = self.output_conv(upsampled)
        
        return output + input_upsampled


class MedicalImageClassifier(nn.Module):
    """v8_batch_normalization: Standard ResNet50 classifier"""
    def __init__(self, num_classes=3):
        super().__init__()
        
        self.backbone = models.resnet50(pretrained=False)
        self.backbone.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        self.classification_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        self.urgency_head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
        self.feature_head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return self.classification_head(features), self.urgency_head(features), self.feature_head(features)


# ============================================================================
# DATASET DEFINITIONS (MATCHING TRAINING STRUCTURE)
# ============================================================================

class EvaluationDataset(Dataset):
    """Dataset for evaluation - loads from category folders"""
    def __init__(self, data_dir, task='classification', max_samples=None):
        self.task = task
        self.data = []
        
        category_map = {'Normal': 0, 'Ischemia': 1, 'Bleeding': 2}
        urgency_map = {'Normal': 0.1, 'Ischemia': 0.7, 'Bleeding': 0.95}
        
        for category, label in category_map.items():
            category_path = os.path.join(data_dir, category, '6_Final_Stripped')
            if os.path.exists(category_path):
                files = [f for f in os.listdir(category_path) if f.endswith('.png')]
                
                # Use last 20% for evaluation (test set)
                split_idx = int(0.8 * len(files))
                eval_files = files[split_idx:]
                
                for filename in eval_files:
                    self.data.append({
                        'path': os.path.join(category_path, filename),
                        'label': label,
                        'urgency': urgency_map[category],
                        'category': category
                    })
        
        if max_samples:
            self.data = self.data[:max_samples]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        img = Image.open(sample['path']).convert('L')
        
        if self.task == 'lapsrn':
            # Create 16x16 LR and 64x64 HR
            hr_size = 64
            lr_size = 16
            hr_img = img.resize((hr_size, hr_size), Image.BICUBIC)
            lr_img = hr_img.resize((lr_size, lr_size), Image.BICUBIC)
            
            lr_array = np.array(lr_img, dtype=np.float32) / 255.0
            hr_array = np.array(hr_img, dtype=np.float32) / 255.0
            
            lr_tensor = torch.from_numpy(lr_array).unsqueeze(0).float()
            hr_tensor = torch.from_numpy(hr_array).unsqueeze(0).float()
            
            return lr_tensor, hr_tensor
            
        elif self.task == 'drrn':
            # Create 64x64 LR and 128x128 HR
            hr_size = 128
            lr_size = 64
            hr_img = img.resize((hr_size, hr_size), Image.BICUBIC)
            lr_img = hr_img.resize((lr_size, lr_size), Image.BICUBIC)
            
            lr_array = np.array(lr_img, dtype=np.float32) / 255.0
            hr_array = np.array(hr_img, dtype=np.float32) / 255.0
            
            lr_tensor = torch.from_numpy(lr_array).unsqueeze(0).float()
            hr_tensor = torch.from_numpy(hr_array).unsqueeze(0).float()
            
            return lr_tensor, hr_tensor
            
        else:  # classification
            # Resize to 224x224 for classifier
            img = img.resize((224, 224), Image.BICUBIC)
            img_array = np.array(img, dtype=np.float32) / 255.0
            img_tensor = torch.from_numpy(img_array).unsqueeze(0).float()
            
            return img_tensor, sample['label'], sample['urgency']


# ============================================================================
# EVALUATION METRICS
# ============================================================================

def calculate_psnr(img1, img2):
    """Calculate Peak Signal-to-Noise Ratio"""
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

def calculate_ssim(img1, img2, window_size=11):
    """Calculate Structural Similarity Index"""
    C1 = 0.01 ** 2
    C2 = 0.03 ** 2
    
    mu1 = F.avg_pool2d(img1, window_size, stride=1, padding=window_size//2)
    mu2 = F.avg_pool2d(img2, window_size, stride=1, padding=window_size//2)
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    
    sigma1_sq = F.avg_pool2d(img1 ** 2, window_size, stride=1, padding=window_size//2) - mu1_sq
    sigma2_sq = F.avg_pool2d(img2 ** 2, window_size, stride=1, padding=window_size//2) - mu2_sq
    sigma12 = F.avg_pool2d(img1 * img2, window_size, stride=1, padding=window_size//2) - mu1_mu2
    
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / \
               ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    
    return ssim_map.mean()

# ============================================================================
# MAIN EVALUATION FUNCTION
# ============================================================================

def evaluate_v8_batch_normalization(data_dir='./preprocessed_data', model_dir='./trained_models_v8/v8_batch_normalization'):
    """
    Evaluate v8_batch_normalization model
    
    Args:
        data_dir: Directory containing preprocessed data with category folders
        model_dir: Directory containing trained model weights
    """
    
    print("="*80)
    print("EVALUATING MODEL: v8_batch_normalization")
    print("="*80)
    print()
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    print()
    
    # Check if data directory exists
    if not os.path.exists(data_dir):
        print(f"ERROR: Data directory '{data_dir}' does not exist!")
        return None
    
    # Load datasets
    print("Loading datasets from category folders...")
    lapsrn_dataset = EvaluationDataset(data_dir, task='lapsrn')
    drrn_dataset = EvaluationDataset(data_dir, task='drrn')
    class_dataset = EvaluationDataset(data_dir, task='classification')
    
    print(f"  LapSRN dataset: {len(lapsrn_dataset)} samples")
    print(f"  DRRN dataset: {len(drrn_dataset)} samples")
    print(f"  Classification dataset: {len(class_dataset)} samples")
    print()
    
    # Create dataloaders
    lapsrn_loader = DataLoader(lapsrn_dataset, batch_size=16, shuffle=False)
    drrn_loader = DataLoader(drrn_dataset, batch_size=16, shuffle=False)
    class_loader = DataLoader(class_dataset, batch_size=16, shuffle=False)
    
    # Load configuration
    config_path = os.path.join(model_dir, 'config.json')
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            config = json.load(f)
        print(f"Loaded configuration for v8_batch_normalization")
        print(f"  Batch Normalization: Enabled")
    else:
        print("Using default v8_batch_normalization configuration")
    print()
    
    # Initialize models
    print("Initializing models...")
    lapsrn = LapSRN(use_batch_norm=True).to(device)
    drrn = DRRN(use_batch_norm=True).to(device)
    classifier = MedicalImageClassifier(num_classes=3).to(device)
    
    # Load model weights
    print("Loading model weights...")
    lapsrn_path = os.path.join(model_dir, 'lapsrn_best.pth')
    drrn_path = os.path.join(model_dir, 'drrn_best.pth')
    classifier_path = os.path.join(model_dir, 'classifier_best.pth')
    
    if not all([os.path.exists(p) for p in [lapsrn_path, drrn_path, classifier_path]]):
        print("ERROR: Model weight files not found!")
        print(f"Expected files in: {model_dir}")
        print("  - lapsrn_best.pth")
        print("  - drrn_best.pth")
        print("  - classifier_best.pth")
        return None
    
    lapsrn.load_state_dict(torch.load(lapsrn_path, map_location=device))
    drrn.load_state_dict(torch.load(drrn_path, map_location=device))
    classifier.load_state_dict(torch.load(classifier_path, map_location=device))
    
    lapsrn.eval()
    drrn.eval()
    classifier.eval()
    print("Models loaded successfully!")
    print()
    
    # ========================================================================
    # EVALUATE LAPSRN
    # ========================================================================
    print("="*80)
    print("Evaluating LapSRN with BatchNorm (16x16 -> 64x64)...")
    print("="*80)
    
    lapsrn_psnr_list = []
    lapsrn_ssim_list = []
    
    with torch.no_grad():
        for lr, hr in tqdm(lapsrn_loader, desc="LapSRN Evaluation"):
            lr = lr.to(device)
            hr = hr.to(device)
            
            sr, _ = lapsrn(lr)
            
            # Calculate metrics
            for i in range(sr.size(0)):
                psnr = calculate_psnr(sr[i:i+1], hr[i:i+1])
                ssim = calculate_ssim(sr[i:i+1], hr[i:i+1])
                lapsrn_psnr_list.append(psnr.item())
                lapsrn_ssim_list.append(ssim.item())
    
    lapsrn_psnr = np.mean(lapsrn_psnr_list)
    lapsrn_ssim = np.mean(lapsrn_ssim_list)
    
    print(f"\nLapSRN Results:")
    print(f"  PSNR: {lapsrn_psnr:.4f} dB")
    print(f"  SSIM: {lapsrn_ssim:.4f}")
    print()
    
    # ========================================================================
    # EVALUATE DRRN
    # ========================================================================
    print("="*80)
    print("Evaluating DRRN with BatchNorm (64x64 -> 128x128)...")
    print("="*80)
    
    drrn_psnr_list = []
    drrn_ssim_list = []
    
    with torch.no_grad():
        for lr, hr in tqdm(drrn_loader, desc="DRRN Evaluation"):
            lr = lr.to(device)
            hr = hr.to(device)
            
            sr = drrn(lr)
            
            # Calculate metrics
            for i in range(sr.size(0)):
                psnr = calculate_psnr(sr[i:i+1], hr[i:i+1])
                ssim = calculate_ssim(sr[i:i+1], hr[i:i+1])
                drrn_psnr_list.append(psnr.item())
                drrn_ssim_list.append(ssim.item())
    
    drrn_psnr = np.mean(drrn_psnr_list)
    drrn_ssim = np.mean(drrn_ssim_list)
    
    print(f"\nDRRN Results:")
    print(f"  PSNR: {drrn_psnr:.4f} dB")
    print(f"  SSIM: {drrn_ssim:.4f}")
    print()
    
    # ========================================================================
    # EVALUATE CLASSIFIER
    # ========================================================================
    print("="*80)
    print("Evaluating Classifier...")
    print("="*80)
    
    all_preds = []
    all_labels = []
    all_urgency_preds = []
    all_urgency_true = []
    
    with torch.no_grad():
        for images, labels, urgency in tqdm(class_loader, desc="Classifier Evaluation"):
            images = images.to(device)
            labels = labels.to(device)
            urgency = urgency.to(device)
            
            class_logits, urgency_pred, _ = classifier(images)
            preds = torch.argmax(class_logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_urgency_preds.extend(urgency_pred.squeeze().cpu().numpy().tolist() if urgency_pred.squeeze().dim() > 0 else [urgency_pred.squeeze().cpu().item()])
            all_urgency_true.extend(urgency.cpu().numpy().tolist() if urgency.dim() > 0 else [urgency.cpu().item()])
    
    # Classification metrics
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    cm = confusion_matrix(all_labels, all_preds)
    
    # Urgency metrics
    urgency_mse = np.mean((np.array(all_urgency_preds) - np.array(all_urgency_true)) ** 2)
    urgency_mae = np.mean(np.abs(np.array(all_urgency_preds) - np.array(all_urgency_true)))
    
    print(f"\nClassifier Results:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"\n  Confusion Matrix:")
    print(f"  {cm}")
    print(f"\n  Urgency MSE: {urgency_mse:.4f}")
    print(f"  Urgency MAE: {urgency_mae:.4f}")
    print()
    
    # Per-class metrics
    class_names = ['Normal', 'Ischemia', 'Bleeding']
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    
    print("  Per-class metrics:")
    for i, class_name in enumerate(class_names):
        if str(i) in report:
            print(f"    {class_name}:")
            print(f"      Precision: {report[str(i)]['precision']:.4f}")
            print(f"      Recall: {report[str(i)]['recall']:.4f}")
            print(f"      F1-score: {report[str(i)]['f1-score']:.4f}")
    print()
    
    # ========================================================================
    # SAVE RESULTS
    # ========================================================================
    results = {
        'version': 'v8_batch_normalization',
        'lapsrn': {
            'psnr': float(lapsrn_psnr),
            'ssim': float(lapsrn_ssim)
        },
        'drrn': {
            'psnr': float(drrn_psnr),
            'ssim': float(drrn_ssim)
        },
        'classifier': {
            'accuracy': float(accuracy),
            'urgency_mse': float(urgency_mse),
            'urgency_mae': float(urgency_mae),
            'confusion_matrix': cm.tolist(),
            'per_class_metrics': report
        }
    }
    
    results_path = os.path.join(model_dir, 'evaluation_results.json')
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4)
    
    print(f"Results saved to: {results_path}")
    print()
    print("="*80)
    print("EVALUATION COMPLETE!")
    print("="*80)
    
    return results


if __name__ == "__main__":
    # Run evaluation
    results = evaluate_v8_batch_normalization(
        data_dir='./preprocessed_data',
        model_dir='./trained_models_v8/v8_batch_normalization'
    )

EVALUATING MODEL: v8_batch_normalization

Using device: cuda

Loading datasets from category folders...
  LapSRN dataset: 1329 samples
  DRRN dataset: 1329 samples
  Classification dataset: 1329 samples

Loaded configuration for v8_batch_normalization
  Batch Normalization: Enabled

Initializing models...
Loading model weights...
Models loaded successfully!

Evaluating LapSRN with BatchNorm (16x16 -> 64x64)...


LapSRN Evaluation: 100% 84/84 [00:05<00:00, 15.15it/s]



LapSRN Results:
  PSNR: 27.1589 dB
  SSIM: 0.8579

Evaluating DRRN with BatchNorm (64x64 -> 128x128)...


DRRN Evaluation: 100% 84/84 [00:07<00:00, 11.77it/s]



DRRN Results:
  PSNR: 33.4558 dB
  SSIM: 0.9632

Evaluating Classifier...


Classifier Evaluation: 100% 84/84 [00:05<00:00, 14.44it/s]


Classifier Results:
  Accuracy: 0.9007

  Confusion Matrix:
  [[885   1   0]
 [ 21 203   0]
 [105   5 109]]

  Urgency MSE: 0.0355
  Urgency MAE: 0.1050

  Per-class metrics:

Results saved to: ./trained_models_v8/v8_batch_normalization/evaluation_results.json

EVALUATION COMPLETE!
